In [1]:
from lib.training import train
from lib.data import get_data_chest_x_ray_image
from lib.utils import get_device
from torch import nn 
import torch.optim as opt

/home/matheusmartin/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class ExperimentConfig:
    def __init__(self, name, model_fn, loss_fn, optimizer_fn):
        self.name = name
        self.model_fn = model_fn
        self.loss_fn = loss_fn
        self.optimizer_fn = optimizer_fn

In [3]:
import os
from torch.utils.data import Dataset, DataLoader, Subset
from lib.data import TransformDataset

def prepare_dataloaders(train_dataset : Dataset, val_dataset : Dataset):
    num_workers = max(1,os.cpu_count()-1)
    
    train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=num_workers, pin_memory=True)
    val_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=True, num_workers=num_workers, pin_memory=True)
    return train_dataloader, val_dataloader

def get_my_metrics(device, n_classes):
    import torchmetrics
    from lib.metrics import MetricCollection
    
    metrics = MetricCollection(device=device)
    metrics.register('accuracy', torchmetrics.Accuracy(task='multiclass', num_classes=n_classes))
    metrics.register('precision', torchmetrics.Precision(task='multiclass', num_classes=n_classes, average='macro'))
    metrics.register('recall', torchmetrics.Recall(task='multiclass', num_classes=n_classes, average='macro'))
    metrics.register('f1_score', torchmetrics.F1Score(task='multiclass', num_classes=n_classes, average='macro'))
    
    return metrics

def experiment(config : ExperimentConfig, data_dict, device, metrics, freeze=False, epochs=30, warmup=5, patience=10):
    
    save_path = f'experiments/{config.name}_folder'
    os.makedirs(save_path, exist_ok=True)   
    
    folds = data_dict['folds']
    base_dataset = data_dict['base_dataset']
    train_transform = data_dict['train_transform']
    val_transform = data_dict['val_transform']
    
    for fold_idx, (train_idx, val_idx) in enumerate(folds):
        print(f"\n--- Fold {fold_idx+1} ---")
        
        model = config.model_fn().to(device=device)
        
        if freeze: model.freeze()
        else : model.unfreeze()

        optimizer = config.optimizer_fn(model.parameters())
        loss_fn = config.loss_fn()
        
        train_subset = Subset(base_dataset, train_idx)
        val_subset = Subset(base_dataset, val_idx)

        train_dataset = TransformDataset(train_subset, train_transform)
        val_dataset = TransformDataset(val_subset, val_transform)

        train_dataloader, val_dataloader = prepare_dataloaders(train_dataset, val_dataset)
        
        save_name = f'{config.name}_fold={fold_idx+1}.pt'
        
        history, model = train(
            model, train_dataloader, val_dataloader,
            loss_fn, optimizer,
            save_path=save_path, save_name=save_name,
            device=device, metrics=metrics, verbose=True,
            epochs=epochs, warmup=warmup, patience=patience,
        )

In [4]:
data_dict = get_data_chest_x_ray_image(img_size=(224, 224), kfold=5)    

In [5]:
from lib.models import MyResnet, MyMobileNet, MyDenseNet

device = get_device()
n_classes = len(data_dict['classes'])
metrics = get_my_metrics(device, n_classes)

In [6]:
resnet18_config = ExperimentConfig(
    name="resnet18",
    model_fn=lambda: MyResnet(resnet_version='resnet18', n_classes=n_classes),
    loss_fn=lambda: nn.CrossEntropyLoss(),
    optimizer_fn=lambda params: opt.Adam(params, lr=1e-3)
)      
experiment(resnet18_config, data_dict, device, metrics, freeze=False, epochs=50)


--- Fold 1 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyResnet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 24 | Train Loss: 0.0402 | Train Accuracy: 0.99% | Val Loss/ Min: 0.2274/0.0943 | Val Accuracy: 0.92% | New Best? False:  46%|████▌     | 23/50 [08:41<10:11, 22.66s/it]



--- Fold 2 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyResnet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 34 | Train Loss: 0.0480 | Train Accuracy: 0.98% | Val Loss/ Min: 0.1627/0.0783 | Val Accuracy: 0.94% | New Best? False:  66%|██████▌   | 33/50 [12:08<06:15, 22.09s/it]



--- Fold 3 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyResnet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 20 | Train Loss: 0.0551 | Train Accuracy: 0.98% | Val Loss/ Min: 0.1982/0.0825 | Val Accuracy: 0.94% | New Best? False:  38%|███▊      | 19/50 [07:15<11:51, 22.94s/it]



--- Fold 4 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyResnet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 22 | Train Loss: 0.0333 | Train Accuracy: 0.99% | Val Loss/ Min: 0.2247/0.0739 | Val Accuracy: 0.93% | New Best? False:  42%|████▏     | 21/50 [07:55<10:57, 22.66s/it]



--- Fold 5 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyResnet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 29 | Train Loss: 0.0555 | Train Accuracy: 0.98% | Val Loss/ Min: 0.0933/0.0693 | Val Accuracy: 0.96% | New Best? False:  56%|█████▌    | 28/50 [10:26<08:12, 22.39s/it]


In [7]:
resnet34_config = ExperimentConfig(
    name="resnet34",
    model_fn=lambda: MyResnet(resnet_version='resnet34', n_classes=n_classes),
    loss_fn=lambda: nn.CrossEntropyLoss(),
    optimizer_fn=lambda params: opt.Adam(params, lr=1e-3)
)
experiment(resnet34_config, data_dict, device, metrics, freeze=False, epochs=50)


--- Fold 1 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyResnet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 17 | Train Loss: 0.0537 | Train Accuracy: 0.98% | Val Loss/ Min: 0.3284/0.1349 | Val Accuracy: 0.88% | New Best? False:  32%|███▏      | 16/50 [06:20<13:29, 23.81s/it]



--- Fold 2 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyResnet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 34 | Train Loss: 0.0351 | Train Accuracy: 0.99% | Val Loss/ Min: 0.5449/0.0727 | Val Accuracy: 0.92% | New Best? False:  66%|██████▌   | 33/50 [12:31<06:26, 22.76s/it]



--- Fold 3 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyResnet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 28 | Train Loss: 0.0364 | Train Accuracy: 0.99% | Val Loss/ Min: 0.1702/0.0855 | Val Accuracy: 0.94% | New Best? False:  54%|█████▍    | 27/50 [10:35<09:01, 23.54s/it]



--- Fold 4 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyResnet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 28 | Train Loss: 0.0690 | Train Accuracy: 0.98% | Val Loss/ Min: 0.1679/0.0676 | Val Accuracy: 0.96% | New Best? False:  54%|█████▍    | 27/50 [10:29<08:56, 23.32s/it]



--- Fold 5 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyResnet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 23 | Train Loss: 0.0570 | Train Accuracy: 0.98% | Val Loss/ Min: 0.1386/0.0726 | Val Accuracy: 0.97% | New Best? False:  44%|████▍     | 22/50 [08:26<10:44, 23.03s/it]


In [8]:
mobilenet_v2_config = ExperimentConfig(
    name="mobilenet_v2",
    model_fn=lambda: MyMobileNet(mobilenet_version='mobilenet_v2', n_classes=n_classes),
    loss_fn=lambda: nn.CrossEntropyLoss(),
    optimizer_fn=lambda params: opt.Adam(params, lr=1e-3)
)
experiment(mobilenet_v2_config, data_dict, device, metrics, freeze=False, epochs=50)


--- Fold 1 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyMobileNet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 23 | Train Loss: 0.0202 | Train Accuracy: 0.99% | Val Loss/ Min: 0.1405/0.0822 | Val Accuracy: 0.97% | New Best? False:  44%|████▍     | 22/50 [08:37<10:58, 23.52s/it]



--- Fold 2 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyMobileNet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 16 | Train Loss: 0.0462 | Train Accuracy: 0.99% | Val Loss/ Min: 0.1084/0.0911 | Val Accuracy: 0.98% | New Best? False:  30%|███       | 15/50 [05:52<13:42, 23.51s/it]



--- Fold 3 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyMobileNet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 31 | Train Loss: 0.0427 | Train Accuracy: 0.98% | Val Loss/ Min: 0.1246/0.0945 | Val Accuracy: 0.97% | New Best? False:  60%|██████    | 30/50 [11:33<07:42, 23.10s/it]



--- Fold 4 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyMobileNet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 30 | Train Loss: 0.0400 | Train Accuracy: 0.99% | Val Loss/ Min: 0.1153/0.0673 | Val Accuracy: 0.97% | New Best? False:  58%|█████▊    | 29/50 [10:47<07:48, 22.31s/it]



--- Fold 5 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyMobileNet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 32 | Train Loss: 0.0195 | Train Accuracy: 0.99% | Val Loss/ Min: 0.3685/0.0722 | Val Accuracy: 0.98% | New Best? False:  62%|██████▏   | 31/50 [11:44<07:11, 22.74s/it]


In [9]:
mobilenet_v3_large_config = ExperimentConfig(
    name="mobilenet_v3_large",
    model_fn=lambda: MyMobileNet(mobilenet_version='mobilenet_v3_large', n_classes=n_classes),
    loss_fn=lambda: nn.CrossEntropyLoss(),
    optimizer_fn=lambda params: opt.Adam(params, lr=1e-3)
)
experiment(mobilenet_v3_large_config, data_dict, device, metrics, freeze=False, epochs=50)


--- Fold 1 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyMobileNet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 35 | Train Loss: 0.0097 | Train Accuracy: 1.00% | Val Loss/ Min: 0.1163/0.1087 | Val Accuracy: 0.97% | New Best? False:  68%|██████▊   | 34/50 [12:46<06:00, 22.54s/it]



--- Fold 2 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyMobileNet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 26 | Train Loss: 0.0158 | Train Accuracy: 0.99% | Val Loss/ Min: 0.1022/0.0791 | Val Accuracy: 0.97% | New Best? False:  50%|█████     | 25/50 [09:31<09:31, 22.88s/it]



--- Fold 3 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyMobileNet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 23 | Train Loss: 0.0088 | Train Accuracy: 1.00% | Val Loss/ Min: 0.1425/0.0910 | Val Accuracy: 0.97% | New Best? False:  44%|████▍     | 22/50 [08:41<11:03, 23.71s/it]



--- Fold 4 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyMobileNet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 34 | Train Loss: 0.0115 | Train Accuracy: 1.00% | Val Loss/ Min: 0.1603/0.0805 | Val Accuracy: 0.98% | New Best? False:  66%|██████▌   | 33/50 [12:28<06:25, 22.68s/it]



--- Fold 5 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyMobileNet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 16 | Train Loss: 0.0341 | Train Accuracy: 0.98% | Val Loss/ Min: 0.1228/0.1113 | Val Accuracy: 0.96% | New Best? False:  30%|███       | 15/50 [05:50<13:37, 23.35s/it]


In [10]:
densenet121_config = ExperimentConfig(
    name="densenet121",
    model_fn=lambda: MyDenseNet(densenet_version='densenet121', n_classes=n_classes),
    loss_fn=lambda: nn.CrossEntropyLoss(),
    optimizer_fn=lambda params: opt.Adam(params, lr=1e-3)
)
experiment(densenet121_config, data_dict, device, metrics, freeze=False, epochs=50)


--- Fold 1 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyDenseNet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 19 | Train Loss: 2.3986 | Train Accuracy: 0.83% | Val Loss/ Min: 569.7438/0.1403 | Val Accuracy: 0.61% | New Best? False:  36%|███▌      | 18/50 [07:33<13:25, 25.17s/it] 



--- Fold 2 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyDenseNet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 30 | Train Loss: 1.1637 | Train Accuracy: 0.86% | Val Loss/ Min: 0.8731/0.3396 | Val Accuracy: 0.79% | New Best? False:  58%|█████▊    | 29/50 [11:44<08:30, 24.29s/it]   



--- Fold 3 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyDenseNet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 20 | Train Loss: 4.5924 | Train Accuracy: 0.80% | Val Loss/ Min: 7.0700/0.3385 | Val Accuracy: 0.85% | New Best? False:  38%|███▊      | 19/50 [07:58<13:00, 25.16s/it]   



--- Fold 4 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyDenseNet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 16 | Train Loss: 0.2490 | Train Accuracy: 0.91% | Val Loss/ Min: 0.4894/0.1216 | Val Accuracy: 0.90% | New Best? False:  30%|███       | 15/50 [06:26<15:01, 25.75s/it]



--- Fold 5 ---
Train Config:
Epochs: 50
Device: cuda
Model Arch: MyDenseNet
Criterion : CrossEntropyLoss
Optimizer : Adam


Epoch 50 | Train Loss: 0.1141 | Train Accuracy: 0.96% | Val Loss/ Min: 0.1575/0.1306 | Val Accuracy: 0.95% | New Best? False: 100%|██████████| 50/50 [19:34<00:00, 23.50s/it]  
